# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察標籤編碼與獨編碼熱的影響

# [作業重點]
- 回答在範例中的觀察結果
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，  
對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?

A:編碼方式的改變對於線性回歸影響超大，因線性回歸目的是為了找出一條線性方程符合整體分佈，但如果特質值為onehot會造成資料很破碎零散，對於線性回歸而言較不適合使用再分類模型上。

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../D7/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)

A:使用onehot確實有助於提升命中率，但相對的會犧牲效能。

In [13]:
# 標籤編碼 + 羅吉斯迴歸

"""
Your Code Here
"""
train_idx = df_train.shape[0]
le = LabelEncoder()
MMS = MinMaxScaler()

df_1 = df.copy()

for i in df.columns:
    df_1[i] = le.fit_transform(df_1[i])

df_le_mms = MMS.fit_transform(df_1)
estimate = LogisticRegression()
start_time = time.time()
print("分數: ", cross_val_score(estimate, df_le_mms[:train_idx], train_Y, cv=10).mean())
print("處理時間: ", time.time() - start_time)

分數:  0.7788764044943821
處理時間:  0.2919807434082031


In [12]:
df_1.shape

(1309, 5)

In [24]:
# 獨熱編碼 + 羅吉斯迴歸

"""
Your Code Here
"""

from sklearn.preprocessing import OneHotEncoder
df_2 = df.copy()

# # way1
# # 使用sklearn方法要注意，如果把整個table拿來做，連同數值型類別都會被轉換成onehot喔~~~
# # MMS 不支援ohe.fit_transform出來的資料型態QQ，故無法使用MinMaxScaler...
# ohe = OneHotEncoder()
# df_2 = ohe.fit_transform(df_1)

# way2
# 注意，pd.get_dummies可以直接把Str轉乘onehot，但是如果是df_1中已經轉換成labelencoder是無法使用pd.get_dummies!!!
# 換句話說，無法轉換數字型態!!!
df_2 = pd.get_dummies(df_2)
MMS = MinMaxScaler() 
df_ohe_mms = MMS.fit_transform(df_2)

estimate = LogisticRegression()
start_time = time.time()
print("分數: ", cross_val_score(estimate, df_ohe_mms[:train_idx], train_Y, cv=10).mean())
print("處理時間: ", time.time() - start_time)

分數:  0.8058302122347065
處理時間:  4.017801523208618


In [22]:
df_2.shape

(1309, 2429)

<h1> 總結來說，使用pandas.get_dummies效能較慢，sk.ohe快上很多。